In [1]:
import pandas as pd
import numpy as np
import openai
from openai.types.beta.threads.message_create_params import (
    Attachment,
    AttachmentToolFileSearch,
)

import picologging as logging

from pydantic import BaseModel, Field

import pathlib
import datetime
import asyncio
from asyncio import Semaphore
import json

import httpx

from typing import Optional, List, Union, Literal, Tuple

In [2]:
with open("../tokens/openai_token.txt") as f:
    openai_token = f.read()

In [3]:
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d_%H%M%S")
pathlib.Path(f"../notebooks_logging/06_ask_q4pages").mkdir(parents=True, exist_ok=True)
# pathlib.Path(f"../notebooks_logging/05_cut_pdfs/{now_str}").mkdir(parents=True, exist_ok=True)

file_handler = logging.FileHandler(f"../notebooks_logging/06_ask_q4pages/{now_str}.log")
stream_handler = logging.StreamHandler()
# stdout_formatter = logging.Formatter(fmt="%(message)s") 
# stream_handler.setFormatter(stdout_formatter)
logging.basicConfig(
    encoding='utf-8', 
    format='%(asctime)s | %(levelname)s | %(message)s', 
    level=logging.INFO, 
    datefmt='%Y-%m-%dT%H:%M:%S',
    handlers=[
        file_handler,
        stream_handler,
    ],
    force=True
)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
li = logger.info
lw = logger.warning
li("start")
lw("check warning")

2025-02-27T03:00:23 | INFO | start
2025-02-27T03:00:23 | WARNING | check warning


In [4]:
df_temp_answers04 = pd.read_parquet(f"../data_temp/04 temp_answers.parquet")
df_temp_answers04 = df_temp_answers04.rename(columns={"documents_to_look_through": "ref_sha1"})
df_temp_answers04

,q_text,q_kind,company_names,company_names_subset,ref_sha1
0,"According to the annual report, what is the Op...",number,[Altech Chemicals Ltd],[Altech Chemicals Ltd],[63688d5d0b4f12e9f847c5407439a1ec46047a4a]
1,"According to the annual report, what is the Op...",number,[Cofinimmo],[Cofinimmo],[9cc771c2171bacc138cda4e7d68b8b427a514d81]
2,Did Cofinimmo outline any new ESG initiatives ...,boolean,[Cofinimmo],[Cofinimmo],[9cc771c2171bacc138cda4e7d68b8b427a514d81]
3,What is the total number of employees let go b...,number,"[Hagerty, Inc.]","[Hagerty, Inc.]",[a6f23184a87f3343f17e8e8ed08f604615cdefc1]
4,Which leadership **positions** changed at Reno...,names,[Renold plc],[Renold plc],[da8afefdc3840175c9f26a4dbbed05e250a342cc]
5,What was the Gross margin (%) for Charles & Co...,number,"[Charles & Colvard, Ltd.]","[Charles & Colvard, Ltd.]",[d3a834539b046a49708161a6c0d35aad29dd15ec]
6,What was the Capital expenditures (in GBP) for...,number,[Harworth Group plc],[Harworth Group plc],[ed55750eae9cb7d893e6484b92496639172717cd]
7,What was the Capital expenditures (in USD) for...,number,"[Charles & Colvard, Ltd.]","[Charles & Colvard, Ltd.]",[d3a834539b046a49708161a6c0d35aad29dd15ec]
8,What are the names of new products launched by...,names,[Zymeworks Inc.],[Zymeworks Inc.],[69b472e4c05986db72ff8e547bb220e2d222b7ef]
9,"For Lipocine Inc., what was the value of Numbe...",number,[Lipocine Inc.],[Lipocine Inc.],[c51f3c5aff7bea6fbc0bb7537838fa2f44f35c23]


In [5]:
df_subset = pd.read_parquet(f"../data_temp/04 subset.parquet")
df_subset

,sha1,cur,company_name,major_industry,mentions_recent_mergers_and_acquisitions,has_leadership_changes,has_layoffs,has_executive_compensation,has_rnd_investment_numbers,has_new_product_launches,...,has_dividend_policy_changes,has_share_buyback_plans,has_capital_structure_changes,mentions_new_risk_factors,has_guidance_updates,has_regulatory_or_litigation_issues,has_strategic_restructuring,has_supply_chain_disruptions,has_esg_initiatives,is_in_questions
0,0a9e39e1d2e176f3a766a0e86af82772f1654a6e,USD,KeyCorp,Financial Services,True,True,False,True,True,True,...,True,True,False,True,True,True,False,False,True,False
1,20c4badd4303f7aba1a298d84be3722fa84e0c67,USD,"Summit Materials, Inc.",Financial Services,True,True,False,True,False,False,...,False,True,False,True,False,False,False,True,True,False
2,30b729c124a24ff21f37431dab6b58dfe7ba56fa,USD,Gibson Energy Inc.,Energy and Utilities,False,False,False,True,False,False,...,True,True,False,False,False,True,False,False,False,False
3,42f03832077a92b3b34855cb7f9ef93563143838,EUR,HCW Biologics Inc.,Healthcare,True,False,False,True,True,True,...,False,False,False,True,True,True,False,False,False,False
4,49a9bf0542f3e5ff0250064bfed4369ecf6c8a09,USD,Essential Metals Limited,Technology,True,False,False,True,True,False,...,False,False,False,True,False,False,False,False,True,False
5,54e625a049a1713eb5a338b64858c06d74e52489,USD,"EVI Industries, Inc.",Retail,True,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,False
6,56262704bde6b584dcebfa644faa23a953498a79,EUR,Koenig & Bauer AG,Technology,True,True,False,True,True,True,...,False,False,False,True,True,False,True,True,True,False
7,58b196e02c2d9749d968a29039e9c2b29d3d31e1,USD,Microchip Technology Incorporated,Technology,True,True,False,True,True,True,...,False,True,True,True,True,True,False,True,True,False
8,5ac3ccdec033f81fab4c2ed9ae86553f4904a450,EUR,"Corbus Pharmaceuticals Holdings, Inc.",Pharmaceuticals,True,False,False,True,True,True,...,False,False,False,True,True,True,False,False,False,False
9,601aba58deffc81230c837404aa883de0d1dde1c,USD,Audalia Resources Limited,Technology,False,True,False,True,True,True,...,False,False,False,True,False,False,False,False,False,True


In [6]:
sem = Semaphore(50)

In [7]:
client = openai.AsyncOpenAI(
    # This is the default and can be omitted
    api_key=openai_token,
)

In [ ]:
for question_id, row in df_temp_answers04.iterrows():
# for question_id, row in df_temp_answers04.iloc[7:,:].iterrows():
            
    li(f"S---------question#{question_id} started")
    li(f"{question_id=}\n{row=}")
    ref_sha1_i = row["ref_sha1"][0]

    pdf_pages = list(pathlib.Path(f"../data_temp/05_cut_pdfs/{ref_sha1_i}/").glob('*'))
    ref_sha1 = row["ref_sha1"]
    for ref_sha1_i in row["ref_sha1"]:
        li(f"S---------question#{question_id} sha1={ref_sha1_i} started")
        pdf_pages = list(pathlib.Path(f"../data_temp/05_cut_pdfs/{ref_sha1_i}/").glob('*'))
        pdf_pages = sorted(pdf_pages)
        li(f'pdf_pages=\n{"\n".join(str(x) for x in pdf_pages)}')
        q_text = row["q_text"]
        li(f"{q_text=}")
        q_kind = row["q_kind"]
        li(f"{q_kind=}")

        match q_kind:
            case "number":
                result_type = 'Union[float, Literal["N/A"]]'
                result_description = 'A float number that is the answer to the question. If the answer is not found then "N/A"'
            case "boolean":
                result_type = 'Union[bool, Literal["N/A"]]'
                result_description = 'A boolean that is the answer to the question. If the answer is not found then "N/A"'
            case "name":
                result_type = 'Union[str, Literal["N/A"]]'
                result_description = 'A string representing a name that is the answer to the question. If the answer is not found then "N/A"'
            case "names":
                result_type = 'Union[List[str], Literal["N/A"]]'
                result_description = 'A list of strings that represent names that are the answer to the question. If the answer is not found then "N/A"'
        str_ans_class = f"""
class Answer(BaseModel):
    result: {result_type} = Field(..., description='{result_description}')
    result_prob: float = Field(..., description="Probability that the result is the right one")
    result_found: bool = Field(..., description="Is the answer to the question present in the file?")

"""

        exec(str_ans_class)
        # li(str_ans_class)

        prompt = f"""Please answer the following question based on the file provided: 
{q_text}

Check that the data is really present in the file.
Check that the found data cannot be confused with other things.
If the question is about currency amount then provide the answer in the currency units and not in thousands or millions. Check which units of measurement are given in the file and convert these units of measurements to currency units. 

---
Output format:
The output should start with @@@ and end with @@@. If the result is empty it should still correspond to the JSON schema and the result should have N/A in result.
The output is a JSON that corresponds to the following pydantic class:
{str_ans_class}
        """

        li(f"pdf_sha1={ref_sha1_i}")
        li(f"q_text={q_text}")
        li(f"q_kind={q_kind}")
        li(f"prompt=\n{prompt}")

        async def send_one_page(page_i: int, pdf_page: str, prompt: str) -> openai.types.beta.threads.Run:
            async with sem:
                try:
                    pdf_assistant = await client.beta.assistants.create(
                        model="gpt-4o",
                        # model="o1-mini",
                        description="An assistant to extract the contents of PDF files.",
                        tools=[{"type": "file_search"}],
                        name="PDF assistant",
                    )
                except (httpx.HTTPError, openai.OpenAIError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d

                # li(f"Assistent created")

                # Create thread
                try:
                    thread = await client.beta.threads.create()
                except (httpx.HTTPError, openai.OpenAIError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d

                # li(f"thread created")
                try:
                    file = await client.files.create(file=open(pdf_page, "rb"), purpose="assistants")
                except (httpx.HTTPError, openai.OpenAIError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d
                # li(f"file created")

                # Create assistant
                try:
                    await client.beta.threads.messages.create(
                        thread_id=thread.id,
                        role="user",
                        attachments=[
                            Attachment(
                                file_id=file.id, tools=[AttachmentToolFileSearch(type="file_search")]
                            )
                        ],
                        content=prompt,
                    )
                except (httpx.HTTPError, openai.OpenAIError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d
                # li(f"message created")

                # Run thread
                try:
                    run = await client.beta.threads.runs.create_and_poll(
                        thread_id=thread.id, assistant_id=pdf_assistant.id, timeout=1000, poll_interval_ms = 3000
                    )
                except (httpx.HTTPError, openai.OpenAIError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d

                messages_cursor = await client.beta.threads.messages.list(thread_id=run.thread_id)
                messages = [message async for message in messages_cursor]

                # Output text
                ans = messages[0].content[0].text.value

                ans_l_splitted = ans.split('@@@')
                if len(ans_l_splitted) == 3:
                    ans_splitted = ans_l_splitted[1]
                    try:
                        ans_d = json.loads(ans_splitted)
                        _ = Answer(**ans_d)
                    except:
                        ans_d = {
                            "result": ans_splitted,
                            "result_error": True
                        }
                else:
                    ans_d = {
                        "result": ans,
                        "result_error": True
                    }
            # if random.uniform(0, 1) < 0.1:
            #     ans_d = {
            #         "result": ans,
            #         "result_error": True
            #     }
            # return question_i, question, ans_d
                return page_i, pdf_page, ans_d
            
        tasks_done = ["0"]*len(pdf_pages)
        tasks_tries = {}
        tasks = set()
        li(f"Sending for one question started")
        for i, pdf_page in enumerate(pdf_pages):
            tasks_tries[i] = 1
            tasks.add(asyncio.create_task(send_one_page(i, pdf_page, prompt)))

        tasks_run = []
        while tasks:
            done, tasks = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
            for completed in done:
                i, param1, ans_d = await completed
                if "result_error" in ans_d:
                    
                    if tasks_tries[i] < 20:
                        if tasks_tries[i] > 5:
                            await asyncio.sleep(1)
                        lw(f"Task {i} failed. Total retries: {tasks_tries[i]}. Retrying")
                        tasks_tries[i] = tasks_tries[i] + 1
                        tasks.add(asyncio.create_task(send_one_page(i, param1, prompt)))
                    else:
                        lw(f"Task {i} failed. Total retries: {tasks_tries[i]}. Stop retrying. Returning the current answer with error.")
                        tasks_done[i] = "2"
                        tasks_run.append((i, param1, ans_d))
                        li("".join(tasks_done))
                else:
                    tasks_done[i] = "1"
                    tasks_run.append((i, param1, ans_d))
                    li("".join(tasks_done))

        tasks_run_sorted = sorted(tasks_run, key=lambda tup: tup[0])
        temp_answers = [ans_d for _, _, ans_d in tasks_run_sorted]
        df_temp_answers = pd.DataFrame(temp_answers)
        li("\n" + str(df_temp_answers))

        answer = {
            "question_id": question_id,
            "q_text": q_text,
            "ref_sha1": list(ref_sha1),
            "ref_sha1_i": ref_sha1_i,
            "ref_sha1_i_page": -1,
            "value": "N/A"
        }

        df_answers_filtered = df_temp_answers[df_temp_answers["result_found"] == True]
        if len(df_answers_filtered)>0:
            if q_kind == "names":
                df_temp_answers["result_str"] = df_temp_answers["result"].astype(str)
                df_answers_filtered = df_temp_answers[df_temp_answers["result_found"] == True]
                v_c_dict = df_answers_filtered["result_str"].value_counts().sort_values(ascending=False).to_dict()
            else:
                v_c_dict = df_answers_filtered["result"].value_counts().sort_values(ascending=False).to_dict()
            value_count_max = next(iter(v_c_dict.items()))
            if value_count_max[1] > 1:
                if q_kind == "names":
                    answer["value"] = df_answers_filtered[df_answers_filtered["result_str"]==value_count_max[0]]["result"].to_list()[0]
                    filter = df_answers_filtered["result_str"]==value_count_max[0]
                else:
                    answer["value"] = value_count_max[0]
                    filter = df_answers_filtered["result"] == value_count_max[0]
                answer["ref_sha1_i_page"] = df_answers_filtered[filter].index.to_list()[0]
            else:
                if q_kind == "number":
                    if len(df_answers_filtered["result"]) % 2 == 0:
                        res_min = df_answers_filtered["result"].min()
                        answer["value"] = df_answers_filtered[df_answers_filtered["result"]!=res_min]["result"].median()
                    else:
                        answer["value"] = df_answers_filtered["result"].median()
                    answer["ref_sha1_i_page"]= df_answers_filtered[df_answers_filtered["result"] == answer["value"]].index.to_list()[0]
                else:
                    answer["value"] = df_answers_filtered["result"].iloc[0]
                    answer["ref_sha1_i_page"] = df_answers_filtered.index.to_list()[0]
        li("\n" + str(df_answers_filtered))
        answer_json_str = json.dumps(answer, indent=4)
        li(answer_json_str)

        filename = f"q{question_id:04d}_{ref_sha1_i}.json"
        folder = pathlib.Path(f"../data_temp/06_ask_questions/")
        folder.mkdir(parents=True, exist_ok=True)
        full_filename = folder/ filename
        with open(full_filename, "w") as f:
            f.write(answer_json_str)

        filename_csv = f"q{question_id:04d}_{ref_sha1_i}.csv"
        filename_parquet = f"q{question_id:04d}_{ref_sha1_i}.parquet"
        folder = pathlib.Path(f"../data_temp/06_ask_questions_df_answers/")
        folder.mkdir(parents=True, exist_ok=True)
        full_filename_csv = folder/ filename_csv
        full_filename_parquet = folder/ filename_parquet
        
        df_answers_filtered.reset_index().to_csv(full_filename_csv, index=False)
        df_answers_filtered_list2str = df_answers_filtered.reset_index().copy()
        if q_kind == "names":
            df_answers_filtered_list2str["result"] = df_answers_filtered_list2str["result"].astype(str)
        df_answers_filtered_list2str.to_parquet(full_filename_parquet, index=False)

        li(f"----------question#{question_id} sha1={ref_sha1_i} finished")
    li(f"----------question#{question_id} finished")

2025-02-27T03:00:23 | INFO | S---------question#0 started
2025-02-27T03:00:23 | INFO | question_id=0
row=q_text                  According to the annual report, what is the Op...
q_kind                                                             number
company_names                                      [Altech Chemicals Ltd]
company_names_subset                               [Altech Chemicals Ltd]
ref_sha1                       [63688d5d0b4f12e9f847c5407439a1ec46047a4a]
Name: 0, dtype: object
2025-02-27T03:00:23 | INFO | S---------question#0 sha1=63688d5d0b4f12e9f847c5407439a1ec46047a4a started
2025-02-27T03:00:23 | INFO | pdf_pages=
..\data_temp\05_cut_pdfs\63688d5d0b4f12e9f847c5407439a1ec46047a4a\63688d5d0b4f12e9f847c5407439a1ec46047a4a_0000.pdf
..\data_temp\05_cut_pdfs\63688d5d0b4f12e9f847c5407439a1ec46047a4a\63688d5d0b4f12e9f847c5407439a1ec46047a4a_0001.pdf
..\data_temp\05_cut_pdfs\63688d5d0b4f12e9f847c5407439a1ec46047a4a\63688d5d0b4f12e9f847c5407439a1ec46047a4a_0002.pdf
..\data_te

CancelledError: 